In [1]:
import pandas as pd
import numpy as np

from scipy import stats

In [2]:
# Load the data from a CSV file
def load_data(filename):

    # Read the CSV file
    data = pd.read_csv(filename)
    # Drop the first column
    data = data.drop(data.columns[0], axis=1)
    # Drop the specified columns to get only numerical data
    nums = data.drop(['subject_id', 'hadm_id', 'charttime'], axis=1)

    return nums

# Calculate mean, variance, and NaN counts for each column
def calculate_stats(data):
    nan_counts = data.isnull().sum()
    smean = data.mean()
    svar = data.var()
    
    return nan_counts, smean, svar

# Fit different distributions to the data and select the best fit
def fit_distributions(data):
    # Define the distributions to be tested
    distributions = {
        'norm': stats.norm,
        'lognorm': stats.lognorm,
        'expon': stats.expon,
        'gamma': stats.gamma,
        'beta': stats.beta,
        'weibull_min': stats.weibull_min,
        't': stats.t,
        'f': stats.f,
    }
    
    results = {}
    test_results = {}
    for column in data.columns:
        column_data = data[column].dropna()
        _, smean, svar = calculate_stats(column_data)  # Extract only mean and variance
        
        best_fit = None
        best_p_value = -1
        test_result = []
        
        for name, distribution in distributions.items():
            try:
                if name in ['lognorm', 'gamma', 'beta', 'weibull_min', 't', 'chi2', 'f', 'pareto']:
                    params = distribution.fit(column_data)
                    ks_stat, p_value = stats.kstest(column_data, name, args=params)
                else:
                    params = distribution.fit(column_data)
                    ks_stat, p_value = stats.kstest(column_data, name, args=params)
                
                test_result.append((name, ks_stat, p_value))
                
                if p_value > best_p_value:
                    best_fit = (name, params)
                    best_p_value = p_value
            except Exception as e:
                print(f"Could not fit {name} distribution to {column}: {e}")
        
        results[column] = best_fit
        test_results[column] = test_result
    
    return results, test_results

# Generate synthetic data based on the best-fitting distributions
def generate_synthetic_data(data, fits):
    samples = len(data)
    synth_data = pd.DataFrame()
    
    for column in data.columns:
        dist_name, params = fits[column]
        distribution = getattr(stats, dist_name)
        synthetic_data = distribution.rvs(*params[:-2], loc=params[-2], scale=params[-1], size=samples)
        
        # Ensure all values are non-negative and cap GCS_Total at 15
        synthetic_data = np.clip(synthetic_data, a_min=0, a_max=None)
        if column == 'GCS Total':
            synthetic_data = np.clip(synthetic_data, a_min=0, a_max=15)
        
        # Introduce NaNs based on the original NaN distribution
        nan_counts = data[column].isnull().sum()
        nan_indices = np.random.choice(samples, nan_counts, replace=False)
        synthetic_data[nan_indices] = np.nan
        
        synth_data[column] = synthetic_data
    
    return synth_data

# Process a file and generate synthetic data
def process_file(filename):
    data = load_data(filename)
    fits, test_results = fit_distributions(data)
    synth_data = generate_synthetic_data(data, fits)

    return data, synth_data, test_results

# Compare statistics of real and synthetic data
def compare_stats(real_data, synth_data):
    real_nan_counts, real_mean, real_var = calculate_stats(real_data)
    synth_nan_counts, synth_mean, synth_var = calculate_stats(synth_data)
    
    print("Comparison of Real and Synthetic Data:\n")
    
    for column in real_data.columns:
        print(f"Column: {column}")
        
        print(f"Real Mean: {real_mean[column]}")
        print(f"Synthetic Mean: {synth_mean[column]}")
        print(f"Mean Difference: {real_mean[column] - synth_mean[column]}\n")
        
        print(f"Real Variance: {real_var[column]}")
        print(f"Synthetic Variance: {synth_var[column]}")
        print(f"Variance Difference: {real_var[column] - synth_var[column]}\n")
        
        print(f"Real NaN Count: {real_nan_counts[column]}")
        print(f"Synthetic NaN Count: {synth_nan_counts[column]}")
        print(f"NaN Count Difference: {real_nan_counts[column] - synth_nan_counts[column]}\n")
        print("-" * 50)

# Print the results of the distribution tests and the best choice
def print_test_results(test_results):
    for column, results in test_results.items():
        print(f"Column: {column}")
        for name, ks_stat, p_value in results:
            print(f"Distribution: {name}, KS Statistic: {ks_stat}, P-Value: {p_value}")
        
        # Determine the best fit
        best_fit = max(results, key=lambda item: item[2])
        print(f"Best Fit: {best_fit[0]}, KS Statistic: {best_fit[1]}, P-Value: {best_fit[2]}")
        print("-" * 50)

In [3]:
# Option 1: Using the earliest chart time for each hadm_id
real_data_op1, synth_data_op1, test_results_op1 = process_file('NumOp1.csv')

#synth_data_op1.head()

c:\Users\Njula Chakaya\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2789: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
c:\Users\Njula Chakaya\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
c:\Users\Njula Chakaya\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_continuous_distns.py:709: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)


In [4]:
#print_test_results(test_results_op1)

In [5]:
#compare_stats(real_data_op1, synth_data_op1)

In [6]:
# Option 2: Picking the charttime with the fewest NaNs
real_data_op2, synth_data_op2, test_results_op2 = process_file('NumOp2.csv')

#synth_data_op2.head()

c:\Users\Njula Chakaya\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2789: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
c:\Users\Njula Chakaya\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_continuous_distns.py:709: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
c:\Users\Njula Chakaya\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
c:\Users\Njula Chakaya\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarn

In [7]:
#print_test_results(test_results_op2)

In [8]:
#compare_stats(real_data_op2, synth_data_op2)

In [9]:
# Option 3: Picking the first reading within the hour (from the start of the first recorded time)

real_data_op3, synth_data_op3, test_results_op3 = process_file('NumOp3.csv')

#synth_data_op3.head()

c:\Users\Njula Chakaya\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2789: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
c:\Users\Njula Chakaya\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
c:\Users\Njula Chakaya\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_continuous_distns.py:709: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)


In [10]:
#print_test_results(test_results_op3)

In [11]:
#compare_stats(real_data_op3, synth_data_op3)